In [ ]:
# !pip freeze >> requirments.txt

In [ ]:
# !pip install fastapi uvicorn
# !pip install  nest-asyncio pyngrok

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive")
# %cd /content/gdrive/MyDrive/Colab Notebooks/skku_python_repo-main/08. 유형별로 실습하는 딥러닝

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/skku_python_repo-main/08. 유형별로 실습하는 딥러닝


# 모델 불러오기

In [ ]:
import pickle
from tensorflow.keras.models import load_model
import pandas as pd

In [ ]:
# 서버 관리용 fastapi 의존 라이브러리
import uvicorn

# fast api 라이브러리
from fastapi import FastAPI

# 인터페이스 데이터 관리를 위한 라이브러리
from pydantic import BaseModel

In [ ]:
from fastapi.middleware.cors import CORSMiddleware
origins = ["*"]

app = FastAPI(title="ML API")

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

In [ ]:
with open("./Injection_Pressure.dump","rb") as fr:
    loadedRef = pickle.load(fr)

In [ ]:
class InDataset(BaseModel):
    inResinTemp : int
    inMoldTemp : int
    inTime : float

In [ ]:
loadedModel = load_model("./Injection_Pressure.h5")

In [ ]:
loadedModel.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 32        
                                                                 
 dense_13 (Dense)            (None, 8)                 72        
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 113 (452.00 Byte)
Trainable params: 113 (452.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
loadedRef

{'temperature': ['RESIN_TEMPERATURE', 'MOLD_TEMPERATURE', 'INJECTION_TIME'],
 'orgtemperaturet': ['RESIN_TEMPERATURE',
  'MOLD_TEMPERATURE',
  'INJECTION_TIME'],
 'label': ['INJECTION_PRESSURE'],
 'model': <keras.src.engine.sequential.Sequential at 0x7da097d907c0>}

In [ ]:
@app.post("/predict", status_code=200)
async def predictDl(x:InDataset):
    print(x)
    # 화면입력데이터 변수 할당
    inResinTemp = x.inResinTemp
    inMoldTemp = x.inMoldTemp
    inTime =x.inTime
    print("step1")
    # 예측을 위한 데이터셋 생성
    TemperatureData = pd.DataFrame( [[ inResinTemp, inMoldTemp, inTime ]] )
    # 예측
    print(TemperatureData)
    predictValue =  int(loadedModel.predict( TemperatureData)[0][0])
    print(predictValue)
    result = {"prediction":predictValue}
    return result

In [ ]:
# inResinTemp = 223
# inMoldTemp = 49
# inTime = 1.9
# print("step2")
# # 예측을 위한 데이터셋 생성
# testData = pd.DataFrame( [[ inResinTemp, inMoldTemp, inTime ]] )
# # 예측
# predictValue = loadedModel.predict( testData)[0][0]
# result = {"prediction":predictValue}
# result

step2
1/1 [==============================] - 0s 63ms/step


{'prediction': 25.351358}

In [ ]:
@app.get("/")
async def root():
    return {"message":"onine"}

In [ ]:
# import nest_asyncio
# from pyngrok import ngrok
# import uvicorn

# auth_token = "2hRjTiUFYINUamvIkMTVUl2hJib_5pMtQqqMBwZP1BDKWyDKn"
# ngrok.set_auth_token(auth_token)
# ngrokTunnel = ngrok.connect(8000)
# print("공용 URL", ngrokTunnel.public_url)
# nest_asyncio.apply()
# uvicorn.run(app, port=8000)

In [ ]:
import uvicorn
if __name__ == "__main__":
  # IP:Injection Pressure (해석 사출압)
  uvicorn.run("ip:app", host="0.0.0.0", port=9999, log_level="debug",
    proxy_headers=True, reload=True)


INFO:     Will watch for changes in these directories: ['/content/gdrive/MyDrive/Colab Notebooks/skku_python_repo-main/08. 유형별로 실습하는 딥러닝']
INFO:     Uvicorn running on http://0.0.0.0:9999 (Press CTRL+C to quit)
INFO:     Started reloader process [251] using WatchFiles
INFO:     Stopping reloader process [251]
